The report clearly describes the learning algorithm, along with the chosen hyperparameters. It also describes the model architectures for any neural networks.
A plot of rewards per episode is included to illustrate that the agent is able to receive an average reward (over 100 episodes) of at least +13. The submission reports the number of episodes needed to solve the environment.
The submission has concrete future ideas for improving the agent's performance.



# Introduction
This is my submission report for Project 1 (Navigation) of Udacity's Deep Reinforcement Learning Nanodegree. In this project, we must train an agent in a Unity environment to collect yellow bananas and avoid blue ones using Deep Q Networks.

### Background and Deep Q Network Overview
In Deep Q Networks (DQNs), we adopt a model-free approach and use a deep neural network to perform function approximation on the action value function. Briefly, our goal is to learn a function Q'(s, a, w), characterised by the states, actions available in each state and a set of learnable weight parameters w, such that Q' approximates Q(s,a), the true action value function. Function approximation is a powerful approach compared to tabular Q-learning because:
1. It enables agents to *generalise*, that is to retrieve action values for states that haven't ever been directly experienced; and,
2. It enables agents to tackle *continuous observation spaces*, which would otherwise require infinitely large Q-tables.  

Deep Neural Networks are excellent non-linear function approximators but previous attempts to combine them with Reinforcement Learning came into significant challenges, since they were found to diverge or become unstable. There are several causes to this instability, two of which I will highlight here:
1. Over learning, sequences of observations can become highly correlated over time-steps. Transitions from one state to the next depend on the action taken by the agent. If an agent learns that a restricted set of actions within a restricted set of states tends to lead to reward, it may well find itself stuck in a loop over transitions involving only those actions and states, which could prevent it from learning the optimal policy; and,
2. The fact that the parameters we're tweaking (weights w) to approximate the action value function are the very same parameters we're using to calculate this value. This creates a situation where we have a target that moves as a direct consequence of and in correlation with our attempts to reach it.  

The oscillations introduced by these two issues are compounded by the fact that small changes to Q may alter the policy significantly. This is because to generate policy from a Q function, for every state we retrieve the action that maximises value: if Q is unstable even by small margins, our policy could thus swing wildly between consecutive iterations.  

In 2015, [Mnih and colleages](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) published an influential paper aiming to overcome these obstacles in combining Reinforcement Learning with Deep Neural Networks. The resulting approach, termed **Deep Q Networks**, relied on two key ideas:
1. **Experience Replay**: learning and experience are dissociated in DQNs. At every time-step, the agent saves its experience to a memory buffer. Every k time-steps, the agent enters a learning epoch wherein it retrieves sequences at random from its memory buffer, which it uses for learning with a temporal difference method. This way, the correlation in experienced sequences is broken; and,
2. **Fixed Q Targets**: to solve the problem of moving targets, the authors created two neural networks - one for learning parameters to approximate the true Q function, and the other for setting targets for this approximation. Weights for setting targets are calculated and updated separately from weights for approximating the Q function, breaking the correlation between them.  

### Algorithm walkthrough
**0.** The agent is initialised with 2 neural networks: a local Q Network for learning weight parameters to approximate the true Q function, and a target Q Network for setting approximation targets.  
**1.** For a series of time-steps T, the agent observes its state, selects an epsilon-greedy action, and is presented by the environment with a reward (or none) and the next state;  
**2.** At the end of each of these time-steps, the agent saves to memory the state, action, reward value and next state resulting from that time-step, which are collectively termed an "experience";  
**3.** Every T time-steps, the agent executes a learning sequence:
    - Sample K experiences at random from the memory buffer;
    - For each experience, use the contents of that experience and i) the target Q Network to estimate the true action value, and ii) the local Q Network to compute its current expectation of action value;
    - Calculates loss (minimum square error) between estimated (target) and currently expected (local) action value;
    - Based on the discrepancy revealed, update the weights of the local Q Network by gradient descent;
    - 'Soft-update' the weights of the target Q Network by a weighted average between them and the local Q Network's weights.
**4.** Return to **1.**

# Algorithm Implementation

## The QNetwork class

We begin by defining a QNetwork class, which instantiates a neural network composed of fully-connected layers according to a pre-defined architecture, where network depth and layer width are specified as arguments. QNetwork objects are neural networks that store and learn the weight parameters necessary for estimating the value of each action available to the agent when given as input a particular state. Accordingly, their input dimensions match the state size, and their output dimensions match the action size. A forward propagation method is defined such that if an instance of a QNetwork is passed a state, it returns the current estimate of value for each action in that state.  
```Python
class QNetwork(nn.Module):
    
    def __init__(self, state_size, action_size, fc1_units=64, fc2_units=64):
        super(QNetwork, self).__init__()
        self.bn1 = nn.BatchNorm1d(num_features=state_size)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)```


## The Agent class and its attributes

We define an Agent class whose only arguments are the state size and action size:
```Python
class Agent():
    
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        
        self.t_step = 0
        
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=lr)
        
        self.memory = ReplayBuffer(action_size, buffer_size, batch_size)
        ```
When initialised with a particular state and action size, the Agent class calls on QNetwork, passes it state size and action size arguments, and instantiates two QNetworks as attributes: a ```qnetwork_local```, which learns to approximate the true action-value function, and a ```qnetwork_target```, which sets the fixed targets for the local QNetwork to learn.  

Agent is also endowed with a ```t_step``` attribute which we will use as a time-step counter for keeping track of how many time-steps have passed since the last time Agent activated a "learning" sequence. In the ```optimizer``` attribute we specify which optimisation method we will use during gradient descent and weight updating.  

To enable Experience Replay, we endow the agent with a ```memory``` attribute, which is an instantiation of the ```ReplayBuffer``` class:
```Python
class ReplayBuffer:
    
    def __init__(self, action_size, buffer_size, batch_size):
        self.action_size = action_size
        self.memory_storage = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=['state', 'action', 'reward', 'next_state', 'done'])```

We use this object to allow the agent to save the state, action, reward, next state and episode status for every time-step. We term the collection of all these variables together for the same time step as an "Experience" tuple. When we initialise a ```ReplayBuffer``` object, we specify an integer for ```buffer_size```, which defines how many Experience tuples the agent's memory can store, and an integer ```batch_size```, which defines how many Experience tuples the agent should 'recall' from memory when performing a learning episode.  

To organise our agent's memory, we create an attribute ```experience``` for the ReplayBuffer class, which we define as a namedtuple object, providing names for it and the fields we expect tuples we pass to it to have.  
The great thing about namedtuple object types is that if we define a namedtuple object *a* it acts as a function, such that if we assign a variable *b* to be the result of passing a tuple of appropriate length through *a*:
```Python
a = namedtuple("Experience", field_names=['state', 'action', 'reward', 'next_state', 'done'])
b=a(1,2,0,4,False)```
then we can access the values of elements in b by calling them through their field names. For example,
```Python
b.done```
returns False. This will be convenient later on, when we recall ```experiences``` from ```memory``` for learning.

## The Agent Class and its methods

### Agent.step, memory.add and memory.sample

```Python
    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)
        
        self.t_step = (self.t_step + 1) % update_every
        
        if self.t_step == 0:
            if len(self.memory) > batch_size:
                experiences = self.memory.sample()
                self.learn(experiences, gamma)```


At every time-step, we will call ```step```, which first calls on the method ```.add```, itself a method of the Agent's ```memory``` and its parent class ReplayBuffer:
```Python
    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory_storage.append(e)```

Calling ```memory.add``` on a (state, action, reward, next_state, done) tuple passes that tuple to ```memory```'s attribute ```experience```, which in turn organises it as a named tuple. That named tuple is then appended to ```memory```'s ```memory_storage``` attribute which is the actual container: a deque of length (capacity) ```buffer_size```.

Method ```step``` then updates the agent's ```t_step``` counter, resetting it to 0 every time ```update_every``` time-steps have passed. If this condition is met *and* there are enough (>```batch_size```) experience tuples stored in memory, the agent calls class ReplayBuffer's method ```.sample```:
```Python
def sample(self):
        experiences = random.sample(self.memory, k = self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)
```
The function of ```.sample``` is to retrieve from memory a number ```k = self.batch_size``` of experiences to learn from, then parse the elements of each experience (state, action, reward, next_state and done booleans) into a convenient form to learn from - that is, to return something as close as possible to a mini-batch that we can feed into a neural network. This is important, because so far our data is organised in a convenient way for retrieving every element of each type in an experience,  whereas in subsequent steps we will want to retrieve elements of a particular type *across all the experiences in a mini-batch*.  

To achieve this, ```.sample``` takes advantage of the namedtuple data structure. Let's exemplify by looking at states. We use list comprehension of ```e.state``` for e in ```experiences``` to retrieve *only* the state component of each of the k experience tuples stored in the variable ```experiences```. We transform the resulting list into a vertical numpy array by passing it to ```np.vstack```, then turn this into a pytorch tensor, with the ```torch.from_numpy``` method. Finally, we cast it as ```.float()``` and send it to GPU so that it's the appropriate format to feed into a neural network for forward propagation. This process is repeated for each of the components of an experience tuple and the return is therefore a tuple of 5 torch tensors.  
           
Having retrieved and re-organised experiences conveniently, ```step``` passes experiences into ```learn```:

### Agent.learn
Having checked that the appropriate number of time-steps (=```update_every```) has been reached and if so, processing Experience tuples into the right format, we are ready to update weights in our networks. We do so by calling the function ```learn```.

```Python
    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences
        
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1) 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        
        loss = F.mse_loss(Q_expected, Q_targets)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.soft_update(self.qnetwork_local, self.qnetwork_target, tau)```
        
First, ```learn``` unpacks the tuple of elements of ```experiences``` each into its own float tensor. Then, in our quest to approximate the action-value function, we need to estimate approximate target values for our local network to learn, as per the formula:

```Python
target_values = rewards + gamma * max(Q_targets_next)```

We split this in two steps for legibility, the first involving forward propagation of next states through the target Q network and retrieval of the maximum value amongst all possible actions:
```Python
Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1) ```
Followed by application of discount factor gamma and adding of observed rewards:
```Python
Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))```  

If an episode terminates at t + 1 there is no next state and therefore the target value should equal reward. An elegant trick to implement this is to invert the tensor of booleans ```dones``` by subtracting it from 1 (True --> False because 1-1=0, False --> True because 1-0=1) then multiplying this by the term ```Q_targets_next```. For episodes which finished at the next state (that is, done = True), 1-dones=0, therefore ```Q_targets = rewards + (gamma * Q_targets_next * 0) <=> Q_targets = rewards + 0```. For episodes whose next state is not done, ```Q_targets = rewards + (gamma * Q_targets_next * 1) <=> Q_targets = rewards + gamma * Q_targets_next```.  

Having obtained our approximated target values, we need to calculate our current estimation of Q, which we do by passing current ```states``` into the ```qnetwork_local``` in order to do forward propagation. Obtaining this value, we can then calculate our mean squared error loss between them. This will tell us how far off the current estimate for action values produced by our local network currently is, and how we need to move weights in order for it to catch up to our target estimate.  

We next execute backpropagation and weight updating by gradient descent, with our chosen optimizer Adam. Notice that we **do not** wish to update weights in the target network with backprop; that is why we call the ```.detach()``` method on this network a couple lines above. ```learn``` is a method that belongs to Agent, not to a specific network, and therefore if we call ```loss.backward()``` this would backpropagate on both networks unless told not to.  

To update weights in the target network, we use ```soft_update``` instead:
```Python
    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)```
What we are trying to do here is to ensure we have different weight parameters for target estimation and approximation. To achieve this, we clone the local network into the target network by copying over in-place all weight parameters, but modifying them slightly by a constant ```tau``` which defines a weighted average of the local model and the target model parameters.

# Training function

Lastly, we implement a training function to call the previous methods and objects in appropriate ways to solve the environment. Function ```dqn``` runs for a ```n_episodes``` number of episodes, and for a ```max_t``` number of time-steps per episode.  

At the beginning of each episode, we reset the environment. Then, for each time-step we call ```agent.act``` to select an action, ```env.step(action)``` to perform that action, receive reward and the next state. After saving these values to appropriate variables we call ```agent.step``` to commit the experience tuple to memory and engage the ```learn``` function or not, depending how many time-steps have passed. We move to the next state by assigning next_state to the state variable, and keep track of episode rewards by adding up reward to ```score```.

```Python    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0] 
        score = 0
        
        for t in range(max_t):
        
            action = int(agent.act(state, eps=eps))
            env_info = env.step(action)[brain_name] 
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)

            state = next_state
            score += reward
            if done:
                break```

At the end of an episode, we append the episode's score to our windowed and by-episode tallies, ```scores_window``` and ```scores``` respectively. We then print the current average score based on the last 100 episodes, and leave on display the previous 100 episode average scores.

```Python
        scores_window.append(score)
        scores.append(score)
        eps = max(eps_end, eps_end*eps)

        print('\rEpisode {}\tAverage Score {:.2f}'.format(i_episode, np.mean(scores_window)), end='')
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))```

At the end of training, we save a checkpoint named after the dqn function input argument ```output```, returning only our episode-by-episode tally:
```Python
    torch.save(agent.qnetwork_local.state_dict(), '{}.pth'.format(output))

    return scores```

# Architecture, hyperparameters and performance
### Architecture
The default values for my QNetwork class define a **2 hidden layer** architecture, where each layer contains **64 units**. I started with this number due to the relatively low size of the input (state size = 37). The network performed well, so I kept this parameter throughout.

I elected also to perform **Batch Normalisation** to normalise the network's input per batch and keep values for the 37 input variables within a similar range. This produced quicker learning and better performance than previous iterations of the network without Batch Normalisation.

### Hyperparameters
My final submission used the following hyperparameters, and I will offer rationale for these choices below.
```Python
buffer_size = 300000
batch_size = 32
gamma = 0.99
tau = 1e-3
lr = 5e-4
update_every = 4```
**Buffer Size = 300,000.** Every time step the Agent will store an experience tuple in this buffer. The training function is defined to run each episode of the task for 300 time-steps and I decided to train for 2,000 episodes. This would yield 600,000 experience tuples. Because ```memory``` is defined as a deque, that means adding new elements beyond its defined capacity will remove older elements one by one. I reasoned that by keeping buffer size below the total number of experiences the Agent would encounter, this would throughout training the agent is losing from its buffer its older experiences, which at that point could be less informative to recall on for learning.

**Batch size = 32.** Batch size defines how many experiences are sampled from memory for learning, setting in effect the mini-batch size for training our neural network, and the minimum number of time-steps that must have elapsed before the agent began learning. Smaller batch sizes will result in noisier estimates of the gradient, whereas larger batches will lead to less episodes of weight updating. 32 appeared as a reasonable compromise. I reasoned batch normalisation could also ameliorate some of the variability problems of choosing a smaller batch size.

**Gamma = 0.99.** The agent is penalised for collecting blue bananas. In choosing its actions, I would therefore like it to learn to anticipate the consequences of its current actions rather than optimise purely immediate reward, as the latter could lead to it taking actions in the present that result in capturing blue bananas in the future.

**Tau = 1e-3.*** Tau is the parameter for weighted averaging between local and target parameters, when soft updating our targets. The higher tau, the less this average weights target parameters. I left this on default setting from the exercises.

**Learning rate = 5e-4.** I kept learning rate as default as well. Using batch normalisation suggests I might have gotten away with increasing learning rate, but the network seemed to be solving the task pretty fast already so I opted not to.

**Update every = 4.** This defines how often to run the learning sequence. I experimented with 8 and 4 and found better results for 4. In effect that means the network is calling learning episodes twice as often, which made it faster at solving the environment.

### Performance

# Future improvements
